In [ ]:
!pip install -q transformers datasets rouge_score accelerate sentencepiece
import torch, pandas as pd
from zipfile import ZipFile
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
)

In [ ]:
ZipFile("/content/drive/MyDrive/data.zip").extractall("/content/data")

In [ ]:
#Load & Inspect Data
train_df = pd.read_csv("/content/data/samsum-train.csv")
val_df   = pd.read_csv("/content/data/samsum-validation.csv")
test_df  = pd.read_csv("/content/data/samsum-test.csv")

print("Train shape:", train_df.shape)
print(train_df.head(2))

Train shape: (14732, 3)
         id                                           dialogue  \
0  13818513  Amanda: I baked  cookies. Do you want some?\r\...   
1  13728867  Olivia: Who are you voting for in this electio...   

                                             summary  
0  Amanda baked cookies and will bring Jerry some...  
1  Olivia and Olivier are voting for liberals in ...  


In [ ]:
#Convert to Hugging Face Dataset
train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)
test_ds  = Dataset.from_pandas(test_df)

**change to SLM here**

In [ ]:
#Tokenization & Preprocessing
from transformers import AutoTokenizer
model_name = "facebook/bart-base"   # you can use bart-large-cnn or t5-base. #chnage here to t-small for change it into SLM , trading power for efficiency
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    inputs = []
    labels = []
    for dialogue, summary in zip(batch["dialogue"], batch["summary"]):
        if isinstance(dialogue, str) and isinstance(summary, str):
            inputs.append(tokenizer(text=dialogue, truncation=True, max_length=512)["input_ids"])
            labels.append(tokenizer(text_target=summary, truncation=True, max_length=128)["input_ids"])
        else:
            # Handle cases where dialogue or summary are not strings, e.g., skip or log a warning
            print(f"Skipping example due to invalid data types: dialogue type = {type(dialogue)}, summary type = {type(summary)}")
            continue # Skip this example

    return {"input_ids": inputs, "labels": labels}


train_tokenized = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
val_tokenized   = val_ds.map(preprocess, batched=True, remove_columns=val_ds.column_names)
test_tokenized  = test_ds.map(preprocess, batched=True, remove_columns=test_ds.column_names)

train_tokenized.set_format(type="torch", columns=["input_ids", "labels"])
val_tokenized.set_format(type="torch", columns=["input_ids", "labels"])
test_tokenized.set_format(type="torch", columns=["input_ids", "labels"])

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Skipping example due to invalid data types: dialogue type = <class 'NoneType'>, summary type = <class 'str'>


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
#Device & Training Config
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [ ]:
#Supervised Fine-tuning (Baseline)
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import torch

# Define training parameters
batch_size = 4
num_epochs = 5
fp16 = torch.cuda.is_available()
model_name = "facebook/a"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/rl_project/baseline",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=fp16,
    logging_dir="/content/drive/MyDrive/rl_project/logs",
    save_total_limit=2,
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator # Add data collator here
)
trainer.train()
model.save_pretrained("/content/drive/MyDrive/rl_project/baseline_model")
tokenizer.save_pretrained("/content/drive/MyDrive/rl_project/baseline_model")

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/tmp/ipython-input-3582275274.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,1.800600,1.552432
2,1.540500,1.491329
3,1.269900,1.485864


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('/content/drive/MyDrive/rl_project/baseline_model/tokenizer_config.json',
 '/content/drive/MyDrive/rl_project/baseline_model/special_tokens_map.json',
 '/content/drive/MyDrive/rl_project/baseline_model/vocab.json',
 '/content/drive/MyDrive/rl_project/baseline_model/merges.txt',
 '/content/drive/MyDrive/rl_project/baseline_model/added_tokens.json',
 '/content/drive/MyDrive/rl_project/baseline_model/tokenizer.json')

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


In [ ]:
from evaluate import load
import torch
rouge = load("rouge")

def evaluate(dataset, name="Baseline", max_samples=200):
    preds, refs = [], []

    # Use a smaller subset for speed if needed
    # Ensure max_samples does not exceed dataset size
    num_samples = min(max_samples, len(dataset))
    for example in dataset.select(range(num_samples)):
        # Tokenize and move inputs to the model's device
        inputs = tokenizer(
            example["dialogue"],
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(model.device)

        # Generate summary (disable gradients for speed)
        with torch.no_grad():
            ids = model.generate(
                **inputs,
                max_length=128,
                num_beams=4,  # optional: improves quality
                early_stopping=True
            )

        # Decode predictions
        pred = tokenizer.decode(ids[0], skip_special_tokens=True)
        preds.append(pred)
        refs.append(example["summary"])

    # Compute ROUGE scores
    result = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
    # Access the ROUGE-L F1 score directly
    print(f"\n{name} ROUGE-L F1:", result["rougeL"])
    return result

# Evaluate on a small subset of the test set
baseline_scores = evaluate(test_ds)


Baseline ROUGE-L F1: 0.4267814580861652


In [ ]:
#RL Fine-tuning
import torch
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
import numpy as np
from evaluate import load

In [ ]:
# Load ROUGE metric
rouge = load("rouge")

# Load baseline model locally
model_rl = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/rl_project/baseline_model").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/rl_project/baseline_model")

In [ ]:
optimizer = Adam(model_rl.parameters(), lr=5e-6)

In [ ]:
# Reward function
def compute_rougeL(ref, pred):
    return rouge.compute(predictions=[pred], references=[ref], use_stemmer=True)["rougeL"]

In [ ]:
# Single RL step
def train_rl_step(batch_dialogues, batch_refs):
    inputs = tokenizer(
        batch_dialogues, return_tensors="pt",
        padding=True, truncation=True, max_length=512
    ).to("cuda")

    # Greedy baseline
    with torch.no_grad():
        greedy_ids = model_rl.generate(**inputs, max_length=128)
    greedy_txt = tokenizer.batch_decode(greedy_ids, skip_special_tokens=True)

    # Sampled output
    sampled_ids = model_rl.generate(
        **inputs, do_sample=True, top_k=50, top_p=0.95, max_length=128
    )
    sampled_txt = tokenizer.batch_decode(sampled_ids, skip_special_tokens=True)

    # Compute advantages
    r_sampled = [compute_rougeL(r, s) for r, s in zip(batch_refs, sampled_txt)]
    r_greedy  = [compute_rougeL(r, g) for r, g in zip(batch_refs, greedy_txt)]
    adv = torch.tensor(np.array(r_sampled) - np.array(r_greedy), dtype=torch.float, device="cuda")

    # Compute log-prob loss
    labels = sampled_ids.clone()
    outputs = model_rl(**inputs, labels=labels)
    log_probs = -outputs.loss
    loss = -(adv * log_probs).mean()

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model_rl.parameters(), 1.0)
    optimizer.step()
    return loss.item()


In [ ]:
# RL Training Loop (demo)
num_epochs = 1      # demo loop
num_batches = 50    # small number for quick run
batch_size = 4

for epoch in range(num_epochs):
    total_loss = 0
    for _ in tqdm(range(num_batches)):
        batch = train_df.sample(batch_size)
        loss = train_rl_step(batch["dialogue"].tolist(), batch["summary"].tolist())
        total_loss += loss
    print(f"Epoch {epoch+1} RL loss:", total_loss / num_batches)

100%|██████████| 50/50 [02:48<00:00,  3.37s/it]

Epoch 1 RL loss: 0.06552471210248768


In [ ]:
model_rl.save_pretrained("/content/drive/MyDrive/rl_project/rl_model")
tokenizer.save_pretrained("/content/drive/MyDrive/rl_project/rl_model")
print("✅ RL model saved locally at /content/drive/MyDrive/rl_project/rl_model")

✅ RL model saved locally at /content/drive/MyDrive/rl_project/rl_model


In [ ]:
# Load RL model
model_rl = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/rl_project/rl_model").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/rl_project/rl_model")

# Reuse your evaluate() function
rl_scores = evaluate(test_ds, name="RL")


RL ROUGE-L F1: 0.4267814580861652


In [ ]:
for i in range(5):
    dialogue = test_df.iloc[i]["dialogue"]
    reference = test_df.iloc[i]["summary"]

    inputs = tokenizer(dialogue, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    summary_ids = model_rl.generate(**inputs, max_length=128, num_beams=4)
    pred_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"Dialogue #{i+1}:\n{dialogue}\n")
    print(f"Reference Summary:\n{reference}\n")
    print(f"RL Model Summary:\n{pred_summary}\n")
    print("-" * 80)


Dialogue #1:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

RL Model Summary:
Amanda can't find Betty's number. Larry called her last time they were at the park together.

--------------------------------------------------------------------------------
Dialogue #2:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll chec

In [ ]:
#Evaluate RL vs Baseline
from evaluate import load
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load ROUGE metric
rouge = load("rouge")

# Re-load tokenizer (same one for both models)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/rl_project/baseline_model")

def full_evaluate(model, name="Model", max_samples=200):
    preds, refs = [], []
    model.eval()

    # Use subset for speed if needed
    num_samples = min(max_samples, len(test_ds))

    for example in test_ds.select(range(num_samples)):
        inputs = tokenizer(
            example["dialogue"],
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(model.device)

        with torch.no_grad():
            ids = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

        pred = tokenizer.decode(ids[0], skip_special_tokens=True)
        preds.append(pred)
        refs.append(example["summary"])

    # Compute ROUGE metrics
    result = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
    scores = {
        "ROUGE-1": round(result["rouge1"], 4),
        "ROUGE-2": round(result["rouge2"], 4),
        "ROUGE-L": round(result["rougeL"], 4),
    }
    print(f"\n{name} Results:", scores)
    return scores


# ✅ Load both models
baseline_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/rl_project/baseline_model").to("cuda")
rl_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/rl_project/rl_model").to("cuda")

# ✅ Evaluate both
baseline_scores = full_evaluate(baseline_model, "Baseline")
rl_scores = full_evaluate(rl_model, "RL")

# ✅ Compare results
pd.DataFrame([baseline_scores, rl_scores], index=["Baseline", "RL"])


Baseline Results: {'ROUGE-1': np.float64(0.508), 'ROUGE-2': np.float64(0.2572), 'ROUGE-L': np.float64(0.4268)}

RL Results: {'ROUGE-1': np.float64(0.5157), 'ROUGE-2': np.float64(0.2561), 'ROUGE-L': np.float64(0.4239)}


,ROUGE-1,ROUGE-2,ROUGE-L
Baseline,0.5080,0.2572,0.4268
RL,0.5157,0.2561,0.4239


In [ ]:
#Improved RL Fine-Tuning Loop (Stable SCST)
import torch
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
import numpy as np
import math

# Load model from your baseline checkpoint
model_rl = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/rl_project/baseline_model").to("cuda")
optimizer = Adam(model_rl.parameters(), lr=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Reward function (already adapted for evaluate.load)
def compute_rougeL(ref, pred):
    return rouge.compute(predictions=[pred], references=[ref], use_stemmer=True)["rougeL"]

def train_rl_step(batch_dialogues, batch_refs, debug=False):
    # Tokenize batch
    inputs = tokenizer(
        batch_dialogues,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # ====== Generate Outputs ======
    # Greedy (baseline)
    with torch.no_grad():
        greedy_ids = model_rl.generate(**inputs, max_length=128)
    greedy_txt = tokenizer.batch_decode(greedy_ids, skip_special_tokens=True)

    # Sampled (exploration)
    sampled_ids = model_rl.generate(
        **inputs, do_sample=True, top_k=50, top_p=0.95, max_length=128
    )
    sampled_txt = tokenizer.batch_decode(sampled_ids, skip_special_tokens=True)

    # ====== Compute Rewards ======
    r_sampled = [compute_rougeL(r, s) for r, s in zip(batch_refs, sampled_txt)]
    r_greedy  = [compute_rougeL(r, g) for r, g in zip(batch_refs, greedy_txt)]
    adv = torch.tensor(np.array(r_sampled) - np.array(r_greedy), dtype=torch.float, device="cuda")

    # Debug print
    if debug:
        print(f"Sampled ROUGE: {r_sampled}")
        print(f"Greedy ROUGE:  {r_greedy}")
        print(f"Advantage:     {adv.tolist()}")

    # Skip if advantage signal is flat
    if torch.allclose(adv, torch.zeros_like(adv)):
        return 0.0, 0.0

    # ====== Compute Log Probabilities (per-sample) ======
    labels = sampled_ids.clone()
    labels[labels == tokenizer.pad_token_id] = -100  # ignore padding in loss

    outputs = model_rl(**inputs, labels=labels, output_hidden_states=False)
    # CrossEntropyLoss is averaged by default → recompute manually per-sample
    logits = outputs.logits  # shape: [B, T, V]
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()

    # Get log-probs for each token
    log_probs = F.log_softmax(shift_logits, dim=-1)
    vocab_size = log_probs.size(-1)
    shift_labels_flat = shift_labels.view(-1)
    log_probs_flat = log_probs.view(-1, vocab_size)

    # Gather the log-probability for the actual generated token
    token_log_probs = log_probs_flat[torch.arange(shift_labels_flat.size(0)), shift_labels_flat]
    token_log_probs = token_log_probs.view(shift_labels.size())
    token_log_probs = token_log_probs.masked_fill(shift_labels == -100, 0.0)

    # Sum log-probs per sequence → shape [B]
    seq_log_probs = token_log_probs.sum(dim=1)

    # ====== Compute RL Loss ======
    adv = (adv - adv.mean()) / (adv.std() + 1e-8)  # normalize advantages
    loss = -(adv * seq_log_probs).mean()

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model_rl.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

    return loss.item(), adv.mean().item()

# ---------------------------
# 🔁 Training Loop
# ---------------------------

num_epochs = 2       # Try 2–3 for stronger learning
num_batches = 200    # Increase for better reward signal
batch_size = 4

for epoch in range(num_epochs):
    total_loss, total_adv = 0, 0
    for step in tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch = train_df.sample(batch_size)
        loss, adv_mean = train_rl_step(batch["dialogue"].tolist(), batch["summary"].tolist())
        total_loss += loss
        total_adv += adv_mean

        if (step + 1) % 10 == 0:
            print(f"Step {step+1}/{num_batches} | Loss: {total_loss/(step+1):.4f} | Avg Advantage: {total_adv/(step+1):.4f}")

    print(f"\n✅ Epoch {epoch+1} finished | Mean Loss: {total_loss/num_batches:.4f} | Mean Advantage: {total_adv/num_batches:.4f}\n")

# Save improved RL model
model_rl.save_pretrained("/content/drive/MyDrive/rl_project/rl_model_stable")
tokenizer.save_pretrained("/content/drive/MyDrive/rl_project/rl_model_stable")


Epoch 1/2:   5%|▌         | 10/200 [00:27<08:00,  2.53s/it]

Step 10/200 | Loss: -13.5625 | Avg Advantage: 0.0000


Epoch 1/2:  10%|█         | 20/200 [00:51<07:04,  2.36s/it]

Step 20/200 | Loss: -3.4108 | Avg Advantage: 0.0000


Epoch 1/2:  15%|█▌        | 30/200 [01:16<06:45,  2.39s/it]

...

Epoch 2/2:  95%|█████████▌| 190/200 [08:03<00:22,  2.21s/it]

Step 190/200 | Loss: -0.7083 | Avg Advantage: 0.0000


Epoch 2/2: 100%|██████████| 200/200 [08:26<00:00,  2.53s/it]

Step 200/200 | Loss: -0.9948 | Avg Advantage: 0.0000

✅ Epoch 2 finished | Mean Loss: -0.9948 | Mean Advantage: 0.0000



('/content/drive/MyDrive/rl_project/rl_model_stable/tokenizer_config.json',
 '/content/drive/MyDrive/rl_project/rl_model_stable/special_tokens_map.json',
 '/content/drive/MyDrive/rl_project/rl_model_stable/vocab.json',
 '/content/drive/MyDrive/rl_project/rl_model_stable/merges.txt',
 '/content/drive/MyDrive/rl_project/rl_model_stable/added_tokens.json',
 '/content/drive/MyDrive/rl_project/rl_model_stable/tokenizer.json')

In [ ]:
model_rl = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/rl_project/rl_model_stable").to("cuda")

rl_scores = full_evaluate(model_rl, "RL (Stable)", max_samples=200)



RL (Stable) Results: {'ROUGE-1': np.float64(0.5053), 'ROUGE-2': np.float64(0.25), 'ROUGE-L': np.float64(0.4218)}


# **TESTING**


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your fine-tuned RL model
model_path = "/content/drive/MyDrive/rl_project//rl_model"
model_rl = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 📝 Enter your own text here (can be a paragraph, article, or conversation)
custom_text = """
The government has announced a new policy to encourage electric vehicle adoption.
It includes tax incentives, charging infrastructure development, and public awareness campaigns.
Experts believe this could accelerate the transition to sustainable transportation.
"""

# Tokenize input
inputs = tokenizer(
    custom_text,
    return_tensors="pt",
    truncation=True,
    max_length=512
).to("cuda")

# Generate summary
with torch.no_grad():
    summary_ids = model_rl.generate(
        **inputs,
        max_length=128,
        num_beams=4,         # beam search for better quality
        early_stopping=True
    )

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("📰 Original Text:\n", custom_text)
print("\n💡 Generated Summary:\n", summary)


📰 Original Text:
 
The government has announced a new policy to encourage electric vehicle adoption.
It includes tax incentives, charging infrastructure development, and public awareness campaigns.
Experts believe this could accelerate the transition to sustainable transportation.


💡 Generated Summary:
 The government has announced a new policy to encourage electric vehicle adoption. It includes tax incentives, charging infrastructure development and public awareness campaigns.


In [ ]:
for i in range(3):
    ids = model_rl.generate(**inputs, do_sample=True, top_k=50, top_p=0.9, max_length=128)
    print(f"\nSummary {i+1}:", tokenizer.decode(ids[0], skip_special_tokens=True))


Summary 1: The government has announced a new policy to encourage electric vehicle adoption. It includes tax incentives, charging infrastructure development, and public awareness campaigns.

Summary 2: The government has announced a new policy to encourage electric vehicle adoption. It includes tax incentives, charging infrastructure development and public awareness campaigns.

Summary 3: The government has announced a new policy to encourage electric vehicle adoption. It includes tax incentives, charging infrastructure development and public awareness campaigns.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your fine-tuned RL model
model_path = "/content/drive/MyDrive/rl_project/rl_model"
model_rl = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 📝 Enter your own text here (can be a paragraph, article, or conversation)
custom_text = """
Raj had the most exciting summer holidays this year.
 He traveled to Andhra Pradesh with his family, eager to explore new places. Their first stop was Visakhapatnam,
  where Raj loved visiting the beaches and trying water sports.
  He was fascinated by the serene beauty of Araku Valley and spent hours capturing its lush landscapes.
   At Borra Caves, he was amazed by the stalactites and stalagmites.
   Raj also enjoyed tasting the local Andhra cuisine, especially spicy biryani and Andhra sweets.
    He visited historic temples in Vijayawada, soaking in the culture and architecture.
    Riding through the hills and valleys, he felt a sense of adventure every day.
     Evenings were spent enjoying local festivals and interacting with friendly locals.
     By the end of his trip, Raj felt happy and refreshed, cherishing memories that would last a lifetime.
"""

# Tokenize input
inputs = tokenizer(
    custom_text,
    return_tensors="pt",
    truncation=True,
    max_length=512
).to("cuda")

# Generate summary
with torch.no_grad():
    summary_ids = model_rl.generate(
        **inputs,
        max_length=128,
        num_beams=4,         # beam search for better quality
        early_stopping=True
    )

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("📰 Original Text:\n", custom_text)
print("\n💡 Generated Summary:\n", summary)

📰 Original Text:
 
Raj had the most exciting summer holidays this year.
 He traveled to Andhra Pradesh with his family, eager to explore new places. Their first stop was Visakhapatnam,
  where Raj loved visiting the beaches and trying water sports. 
  He was fascinated by the serene beauty of Araku Valley and spent hours capturing its lush landscapes.
   At Borra Caves, he was amazed by the stalactites and stalagmites. 
   Raj also enjoyed tasting the local Andhra cuisine, especially spicy biryani and Andhra sweets.
    He visited historic temples in Vijayawada, soaking in the culture and architecture.
    Riding through the hills and valleys, he felt a sense of adventure every day.
     Evenings were spent enjoying local festivals and interacting with friendly locals. 
     By the end of his trip, Raj felt happy and refreshed, cherishing memories that would last a lifetime.


💡 Generated Summary:
 Raj spent summer holidays in Andhra Pradesh with his family. His first stop was Visakhap

In [ ]:
for i in range(3):
    ids = model_rl.generate(**inputs, do_sample=True, top_k=50, top_p=0.9, max_length=128)
    print(f"\nSummary {i+1}:", tokenizer.decode(ids[0], skip_special_tokens=True))


Summary 1: Raj travelled to Andhra Pradesh with his family, eager to explore new places. His first stop was Visakhapatnam, where he loved visiting the beaches and trying water sports.

Summary 2: Raj travelled to Andhra Pradesh with his family, eager to explore new places. His first stop was Visakhapatnam, where he enjoyed visiting the beaches and trying water sports. He spent a lot of time in the mountains and valleys.

Summary 3: Raj spent summer holidays in Andhra Pradesh with his family. His first stop was Visakhapatnam, where he enjoyed visiting the beaches and trying water sports. He also spent time at Borra Caves.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your fine-tuned RL model
model_path = "/content/drive/MyDrive/rl_project/rl_model"
model_rl = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 📝 Enter your own text here (can be a paragraph, article, or conversation)
custom_text = """
Dear Team,
I hope this email finds you well.
 As discussed in yesterday’s meeting, we need to finalize the project proposal by Friday.
 Please ensure that all sections are updated, especially the financial estimates and timeline.
 Also, share the draft with me by Thursday for review.
Regards,
Manager
"""
def generate_summary(model, tokenizer, text, device="cuda"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    ids = model.generate(
        **inputs,
        num_beams=6,               # try 4,6,8
        length_penalty=1.0,        # 0.6..1.2 (higher favours longer summaries)
        max_length=128,
        no_repeat_ngram_size=3,    # prevents repetition
        early_stopping=True,
        num_return_sequences=1
    )
    return tokenizer.decode(ids[0], skip_special_tokens=True)

# Tokenize input
inputs = tokenizer(
    custom_text,
    return_tensors="pt",
    truncation=True,
    max_length=512
).to("cuda")

# Generate summary
with torch.no_grad():
    summary_ids = model_rl.generate(
        **inputs,
        max_length=512,
        num_beams=4,         # beam search for better quality
        early_stopping=True
    )

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("📰 Original Text:\n", custom_text)
print("\n💡 Generated Summary:\n", summary)

📰 Original Text:
 
Dear Team,
I hope this email finds you well.
 As discussed in yesterday’s meeting, we need to finalize the project proposal by Friday. 
 Please ensure that all sections are updated, especially the financial estimates and timeline. 
 Also, share the draft with me by Thursday for review.
Regards,
Manager


💡 Generated Summary:
 As discussed in yesterday's meeting, they need to finalize the project proposal by Friday.


As discussed in yesterday's meeting, they need to finalize the project proposal by Friday.

In [19]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your fine-tuned RL model
model_path = "/content/drive/MyDrive/rl_project/rl_model"
model_rl = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 📝 Enter your own text here (can be a paragraph, article, or conversation)
custom_text = """
Dear Students,
As part of your coursework, you are required to submit a project report.
The report should be 15 pages minimum, with proper references. Please form groups of 4–5.
The submission deadline is 20th October. Late submissions will not be accepted.
We will also have a viva on 25th October. Kindly ensure originality to avoid plagiarism issues.
"""

# Tokenize input
inputs = tokenizer(
    custom_text,
    return_tensors="pt",
    truncation=True,
    max_length=512
).to("cuda")

# Generate summary
'''with torch.no_grad():
    summary_ids = model_rl.generate(
        **inputs,
        max_length=120,
        min_length=40,
        num_beams=10,         # beam search for better quality
        early_stopping=True
    )'''



summary_ids = model_rl.generate(
    **inputs,
    max_length=120,
    min_length=30,
    num_beams=4,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    do_sample=True,
    early_stopping=True
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("📰 Original Text:\n", custom_text.strip())
print("\n💡 Generated Summary:\n", summary)

📰 Original Text:
 Dear Students,
As part of your coursework, you are required to submit a project report.
The report should be 15 pages minimum, with proper references. Please form groups of 4–5.
The submission deadline is 20th October. Late submissions will not be accepted.
We will also have a viva on 25th October. Kindly ensure originality to avoid plagiarism issues.

💡 Generated Summary:
 As part of your coursework, you are required to submit a project report. The deadline is 20th October. The viva on 25th October is also required.


In [2]:
!pip install flask flask-ngrok


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


In [4]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import pipeline

summarizer = pipeline("summarization", model="/content/drive/MyDrive/rl_project/rl_model")

app = Flask(__name__)
run_with_ngrok(app)  # Enables public access to your Colab Flask app

@app.route('/summarize', methods=['POST'])
def summarize():
    data = request.get_json()
    text = data['text']
    result = summarizer(text, max_length=150, min_length=40, do_sample=False)
    return jsonify({'summary': result[0]['summary_text']})

app.run()

Device set to use cuda:0


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [5]:
!pip install flask pyngrok


In [2]:
!pip install flask pyngrok transformers torch


In [11]:
from pyngrok import ngrok
from flask import Flask, request, jsonify
from transformers import pipeline

# Set your ngrok auth token
ngrok.set_auth_token("34aheODnO9cGcWMVfFdO3vd9FzG_7NWadUxb8hLqhPchdgxXt")


In [12]:
# Create Flask app
app = Flask(__name__)

# Load summarization model
summarizer = pipeline("summarization", model="/content/drive/MyDrive/rl_project/rl_model")

@app.route('/')
def home():
    return "<h2>Text Summarization API is running!</h2><p>Use POST /summarize with JSON {'text': 'your text here'}</p>"

@app.route('/summarize', methods=['POST'])
def summarize():
    data = request.get_json()
    text = data.get('text', '')
    if not text.strip():
        return jsonify({'error': 'No input text provided'}), 400
    summary = summarizer(text, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    return jsonify({'summary': summary})


Device set to use cuda:0


In [13]:
# Open ngrok tunnel on port 5000
public_url = ngrok.connect(5000).public_url
print(f"Your public app URL: {public_url}")

# Run Flask app
app.run(port=5000)


Your public app URL: https://nery-unintoned-gloomily.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Oct/2025 05:55:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2025 05:55:27] "GET /favicon.ico HTTP/1.1" 404 -


In [3]:
from pyngrok import ngrok
from flask import Flask, request, jsonify, render_template
from transformers import pipeline
import os

# Authenticate with ngrok
ngrok.set_auth_token("34aheODnO9cGcWMVfFdO3vd9FzG_7NWadUxb8hLqhPchdgxXt")

# Create Flask app
app = Flask(__name__)

# Load summarization model
summarizer = pipeline("summarization", model="/content/drive/MyDrive/rl_project/rl_model")

@app.route('/')
def home():
    return render_template("index.html")

@app.route('/summarize', methods=['POST'])
def summarize():
    data = request.get_json() or {}
    text = data.get('text', "")
    if not text.strip():
        return jsonify({"error": "No text provided"}), 400

    summary = summarizer(text, max_length=150, min_length=40, do_sample=False)
    return jsonify({"summary": summary[0]['summary_text']})

# Create folders for templates if not present
os.makedirs("templates", exist_ok=True)

# Save HTML frontend file
html_code = '''<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>AI Text Summarizer</title>

  <!-- Google Fonts -->
  <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&family=Inter:wght@400;500;700&display=swap" rel="stylesheet">

  <!-- Styling -->
  <style>
    body {
      margin: 0;
      font-family: 'Poppins', sans-serif;
      background: linear-gradient(135deg, #e1f5fe, #f3e5f5);
      color: #333;
    }

    header {
      text-align: center;
      padding: 40px 20px;
      background: rgba(255, 255, 255, 0.6);
      backdrop-filter: blur(10px);
      box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1);
      border-bottom: 1px solid rgba(0, 0, 0, 0.1);
    }

    header h1 {
      margin: 0;
      font-weight: 700;
      font-size: 2rem;
      color: #3f51b5;
    }

    header p {
      margin-top: 8px;
      color: #555;
      font-size: 1rem;
    }

    main {
      max-width: 800px;
      margin: 50px auto;
      background: #fff;
      padding: 40px;
      border-radius: 10px;
      box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1);
    }

    textarea {
      width: 100%;
      height: 200px;
      padding: 15px;
      font-size: 1rem;
      border: 1px solid #ccc;
      border-radius: 8px;
      outline: none;
      resize: none;
      transition: border-color 0.3s;
    }

    textarea:focus {
      border-color: #3f51b5;
    }

    button {
      margin-top: 20px;
      padding: 12px 30px;
      font-size: 1rem;
      font-weight: 500;
      color: white;
      background: linear-gradient(90deg, #3f51b5, #5c6bc0);
      border: none;
      border-radius: 8px;
      cursor: pointer;
      transition: transform 0.2s, background 0.3s;
    }

    button:hover {
      transform: translateY(-2px);
      background: linear-gradient(90deg, #303f9f, #3949ab);
    }

    .result-box {
      margin-top: 30px;
      background: #f7f9fc;
      border-left: 5px solid #3f51b5;
      padding: 20px;
      border-radius: 8px;
      min-height: 100px;
    }

    footer {
      text-align: center;
      margin-top: 40px;
      padding: 20px;
      color: #666;
      font-size: 0.9rem;
    }

    @media (max-width: 600px) {
      main {
        padding: 25px;
        margin: 20px;
      }

      textarea {
        height: 150px;
      }
    }
  </style>
</head>

<body>
  <header>
    <h1>AI Text Summarizer</h1>
    <p>Generate concise summaries using advanced NLP models</p>
  </header>

  <main>
    <label for="inputText"><strong>Enter your text to summarize:</strong></label>
    <textarea id="inputText" placeholder="Paste or type your paragraph here..."></textarea>
    <button onclick="summarize()">Summarize</button>

    <div class="result-box" id="outputText">
      <em>Summary will appear here...</em>
    </div>
  </main>

  <footer>
    © 2025 AI Summarization App | Powered by Flask & Hugging Face Transformers
  </footer>

  <script>
    async function summarize() {
      const text = document.getElementById('inputText').value.trim();
      const output = document.getElementById('outputText');
      if (!text) {
        output.innerHTML = "<em style='color:red;'>Please enter some text first!</em>";
        return;
      }
      output.innerHTML = "<em>Summarizing, please wait...</em>";
      try {
        const res = await fetch('/summarize', {
          method: 'POST',
          headers: {'Content-Type': 'application/json'},
          body: JSON.stringify({ text: text })
        });
        const data = await res.json();
        if (data.summary) {
          output.innerHTML = `<strong>Summary:</strong><br>${data.summary}`;
        } else {
          output.innerHTML = `<em style='color:red;'>Error: ${data.error || "Unexpected issue occurred."}</em>`;
        }
      } catch (error) {
        output.innerHTML = `<em style='color:red;'>Server Error: Unable to connect.</em>`;
      }
    }
  </script>
</body>
</html>

'''
with open("templates/index.html", "w") as f:
    f.write(html_code)


Device set to use cuda:0


In [4]:
# Connect to ngrok and run Flask
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")
app.run(port=5000)


Public URL: https://nery-unintoned-gloomily.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Oct/2025 10:04:21] "GET / HTTP/1.1" 200 -
Your max_length is set to 150, but your input_length is only 113. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:werkzeug:127.0.0.1 - - [26/Oct/2025 10:06:30] "POST /summarize HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2025 10:06:35] "GET / HTTP/1.1" 200 -
Your max_length is set to 150, but your input_length is only 113